In [1]:
from scboolseq import scBoolSeq

import glob 
import numpy as np
import pandas as pd
import sklearn

/home/pauleve/.local/lib/python3.12/site-packages/scboolseq/binarization.py:46: UserWarning: 
Scikit-learn's transform output is set to `default`
Please consider calling `sklearn.set_config(transform_output='pandas')` to set this option globally.
Otherwise use a config context to conserve DataFrame output
>>>with sklearn.config_context(transform_output='pandas'):
>>>    bin_rna_data = scboolseq.scBoolSeq().fit_transform(log_rna_data)
  warnings.warn(


In [2]:
ground_truth_prefix = "../ground-truth/"
workdir = "_workdir"
background_scRNA_seq = "GSE60361_mouse_brain.csv"
background_scRNA_seq_src = f"https://github.com/bnediction/scBoolSeq-supplementary/raw/main/data_filtered_vargenes/{background_scRNA_seq}"
output_prefix = "../"

SEED = 21382

## Load background scRNA-seq data

In [3]:
background_scRNA_seq_file = f"{workdir}/{background_scRNA_seq}"
! test -f {background_scRNA_seq_file} || (mkdir {workdir} && curl -fL {background_scRNA_seq_src} -o {background_scRNA_seq_file})

In [4]:
ref_data = pd.read_csv(background_scRNA_seq_file, index_col=0)
ref_data.head()

,Atp1b2,Cxcl14,Nacc2,Scg2,Nrxn3,Rph3a,Pcp4l1,Ablim1,Xist,Marcks,...,Hopx,Fxyd1,Id3,Mgp,Acta2,Myl9,Crip1,Sptssa,Dynlt1b,Ctnna1
1772071015-C02,8.489427,8.933823,0.000000,9.535737,8.933823,8.913243,8.128726,7.974452,8.342811,8.422048,...,8.756738,0.0,0.0,0.0,0.0,0.0,0.0,7.426293,0.000000,0.000000
1772071017-G12,8.162866,8.877479,8.925605,8.678864,8.633270,9.202052,8.947434,8.162866,0.000000,8.162866,...,7.460357,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
1772071017-A05,8.014864,8.597981,7.721852,8.821713,9.038983,8.140482,8.597981,7.721852,0.000000,8.278299,...,7.840381,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
1772071014-B06,7.872927,9.092677,8.347984,8.869821,8.646387,8.782817,8.898557,6.394773,8.114747,8.468354,...,7.386175,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6.394773,0.000000
1772067065-H06,8.332319,0.000000,8.478931,9.350934,8.995119,8.675213,8.332319,7.963969,0.000000,9.137178,...,7.415830,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,6.756281


In [5]:
with sklearn.config_context(transform_output="pandas"):
    scbool = scBoolSeq().fit(ref_data)

/usr/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


## Generate sample scRNA-seq from Boolean trajectories

In [6]:
!mkdir -p {output_prefix}traj/
!mkdir -p {output_prefix}steady/

In [6]:
def make_nb_cells(traj_df, nb_cells_transient=(150,250), nb_cells_steady=(500,600), SEED=SEED):
    rng = np.random.default_rng(SEED)
    n_samples = rng.integers(*nb_cells_transient, size=len(traj_df.index))
    _steady = np.where(traj_df.index.map(lambda x: "steady" in x and "_to_" not in x))[0]
    n_samples[_steady] = rng.integers(*nb_cells_steady, size=len(_steady))
    return n_samples

def expand_bindata(traj_df, n_samples):
    d = traj_df.copy(deep=True).values.repeat(n_samples, axis=0)
    return pd.DataFrame(d, columns=traj_df.columns)

def push_mutants_counts(counts, name):
    for label, mutant_counts in counts.groupby(lambda idx: idx.split("#")[0]):
        mutant_counts.index = [idx[idx.index("#")+1:] for idx in mutant_counts.index]
        print(label, name)
        mutant_counts.T.to_csv(f"{output_prefix}traj/{label}-{name}.csv")
        sel = [i for i in mutant_counts.index if i.startswith("steady")]
        mutant_counts.loc[sel].T.to_csv(f"{output_prefix}steady/{label}-{name}.csv")

def make_mutant_counts(traj_df, n_samples, SEED=SEED):
    bindata = expand_bindata(traj_df, n_samples)
    for args, name in [({}, "normalized-scRNAseq-dropouts"), 
                       ({"dropout_mode": None}, "normalized-scRNAseq-nodropouts")]:
        counts = scbool.sample_counts(bindata, n_samples_per_state=1, random_state=SEED)
        counts.index = [f"{x}_{y}" for i,x in enumerate(traj_df.index) for y in range(n_samples[i])]
        counts.index.name = "cellID"
        push_mutants_counts(counts, name)

In [7]:
_suffix = "-boolean-trajectories.csv"
def pull_traj_df(traj_file):
    label = traj_file[len(ground_truth_prefix):-len(_suffix)]
    traj_df = pd.read_csv(traj_file, index_col=0)
    traj_df.index = [f"{label}#{i}" for i in traj_df.index]
    return traj_df

trajs_df = pd.concat([pull_traj_df(traj_file) for traj_file in glob.glob(f"{ground_truth_prefix}*{_suffix}")])
n_samples = make_nb_cells(trajs_df)
make_mutant_counts(trajs_df, n_samples)

/home/pauleve/.local/lib/python3.12/site-packages/scboolseq/simulation.py:862: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Skewness": ss.skew(trajectory),
/home/pauleve/.local/lib/python3.12/site-packages/scboolseq/simulation.py:863: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Kurtosis": ss.kurtosis(trajectory),
/usr/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


gene10KO normalized-scRNAseq-dropouts
gene1KO normalized-scRNAseq-dropouts
gene2KO normalized-scRNAseq-dropouts
gene3KO normalized-scRNAseq-dropouts
gene4KO normalized-scRNAseq-dropouts
gene5KO normalized-scRNAseq-dropouts
gene6KO normalized-scRNAseq-dropouts
gene7KO normalized-scRNAseq-dropouts
gene8KO normalized-scRNAseq-dropouts
gene9KO normalized-scRNAseq-dropouts
wt normalized-scRNAseq-dropouts


/home/pauleve/.local/lib/python3.12/site-packages/scboolseq/simulation.py:862: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Skewness": ss.skew(trajectory),
/home/pauleve/.local/lib/python3.12/site-packages/scboolseq/simulation.py:863: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  "Kurtosis": ss.kurtosis(trajectory),
/usr/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


gene10KO normalized-scRNAseq-nodropouts
gene1KO normalized-scRNAseq-nodropouts
gene2KO normalized-scRNAseq-nodropouts
gene3KO normalized-scRNAseq-nodropouts
gene4KO normalized-scRNAseq-nodropouts
gene5KO normalized-scRNAseq-nodropouts
gene6KO normalized-scRNAseq-nodropouts
gene7KO normalized-scRNAseq-nodropouts
gene8KO normalized-scRNAseq-nodropouts
gene9KO normalized-scRNAseq-nodropouts
wt normalized-scRNAseq-nodropouts
